In [1]:
import numpy as np

In [2]:
class ANN:
    def __init__(self, 
                 layer_size= [50,1], 
                 lr= 0.001, 
                 momentum= 0, 
                 batch_size= None, 
                 init_seed = None, 
                 nb_epoch= 300):
        self.x = None
        self.y = None
        self.lr = lr
        self.momentum = momentum
        self.layer_size = layer_size
        self.weights = []
        self.biases = []
        self.velocity = []
        self.velocity_bias = []
        self.output_layer = []
        self.batch_size = batch_size
        
        self.nb_epoch = nb_epoch
        
        if init_seed != None:
            np.random.seed(init_seed)
            
        for i, layer in enumerate(layer_size):
            if i < (len(layer_size) - 1):
                limit = np.sqrt(6 / (layer_size[i] + layer_size[i+1]))
                self.weights.append(np.random.rand(layer_size[i], layer_size[i+1]) * 2 * limit - limit)
                self.velocity.append(np.zeros((layer_size[i], layer_size[i+1])))
            limit = np.sqrt(6 / (layer_size[i] + 1))
            self.biases.append(np.random.rand(layer_size[i]) * 2 * limit - limit)
            self.velocity_bias.append(np.zeros(layer_size[i]))
        
    def _sigmoid(self, x, derivative= False):
        sigmoid =  1 / (1 + np.exp(-x))
        if derivative:
            return sigmoid * (1 - sigmoid)
        else:
            return sigmoid
    
    def fit(self, x, y):
        self.x = np.array(x, dtype= np.float32)
        self.y = np.array(y, dtype= np.float32)
        if len(self.y.shape) == 1:
            self.y = np.array([[y] for y in self.y], dtype= np.float32)
        
        limit = np.sqrt(6 / (self.x.shape[1] + self.layer_size[0]))
        self.weights = [np.random.rand(self.x.shape[1], self.layer_size[0]) * limit * 2 - limit] + self.weights
        self.velocity = [np.zeros((self.x.shape[1], self.layer_size[0]))] + self.velocity
        if self.batch_size is None:
            self.batch_size = self.x.shape[0]
        
        for i in range(self.nb_epoch):
            loss = 0
            indices = np.arange(self.y.shape[0])
            np.random.shuffle(indices)            
            self.x = self.x[indices]
            self.y = self.y[indices]
            
            for idx in range(0, self.x.shape[0], self.batch_size):
                data_in = self.x[idx : min(idx + self.batch_size, self.x.shape[0])]
                data_target = self.y[idx : min(idx + self.batch_size, self.x.shape[0])]
                out = self._feed_forward(data_in)
                loss += np.sum(self._error(out, data_target))
                self._backprop(data_in, data_target)
            print("Epoch {}, loss: {}".format(i, loss))
    
    def _feed_forward(self, input_data= []):
        self.output_layer = []
        in_layer = input_data
        for weight, bias in zip(self.weights, self.biases):
            out_layer = self._sigmoid(np.dot(in_layer, weight) + bias)
            self.output_layer.append(out_layer)
            
            in_layer = out_layer
        return out_layer
        
    def _error(self, pred, actual, derivative = False): 
        if derivative:
            return (actual - pred) / (pred - (pred ** 2))
        else :
            return -(actual * np.log(pred) + ((1-actual) * np.log(1-pred)))
    
    def _backprop(self, data_in, data_target):
        for i, w in reversed(list(enumerate(self.weights))):
            if i == len(self.weights) - 1:
                out_d = self._error(self.output_layer[i], data_target, derivative=True) * self._sigmoid(self.output_layer[i], derivative= True)
                d_weight = self.lr * np.dot(self.output_layer[i - 1].T, out_d) + self.velocity[i] * self.momentum
            elif i == 0:
                out_d = np.dot(out_d, self.weights[i + 1].T) * self._sigmoid(self.output_layer[i], derivative= True)
                d_weight = self.lr * np.dot(data_in.T, out_d) + self.velocity[i] * self.momentum
            else:
                out_d = np.dot(out_d, self.weights[i + 1].T) * self._sigmoid(self.output_layer[i], derivative= True)
                d_weight = self.lr * np.dot(self.output_layer[i - 1].T, out_d) + self.velocity[i] * self.momentum
#             print(out_d.shape)
            d_bias = self.lr * np.sum(out_d, axis= 0) + self.velocity_bias[i] * self.momentum
            
            self.velocity_bias[i] = d_bias
            self.velocity[i] = d_weight
        
        for i, w in enumerate(self.weights):
#             print("updating layer {}".format(i))
            self.weights[i] += self.velocity[i]
#             print(self.biases[i].shape)
#             print(self.velocity_bias[i].shape)
            self.biases[i] += self.velocity_bias[i]
            
    
    def predict(self, data):
        data = np.array(data, dtype= np.float32)
        return self._feed_forward(data)
    
    def get_weights(self):
        return self.weights
    
    def get_biases(self):
        return self.biases

## Experiment 1
Data point

In [6]:
ann = ANN(layer_size=[4, 4, 1], momentum= 0.9, lr= 0.05, batch_size=2, nb_epoch= 30)
x = [[0,1], [0,0], [1,0], [1,1]]
y = [[0], [0], [1], [1]]


ann.fit(x, y)
ann.predict([[0,0], [0,0.1], [10.1,0]])

Epoch 0, loss: 3.640343595735515
Epoch 1, loss: 3.1185709434955973
Epoch 2, loss: 2.88736671847234
Epoch 3, loss: 2.7767926314669165
Epoch 4, loss: 2.7895219367377666
Epoch 5, loss: 2.8467273969890843
Epoch 6, loss: 3.013042656913342
Epoch 7, loss: 2.9026082732157237
Epoch 8, loss: 2.993399730421239
Epoch 9, loss: 2.9250136230586605
Epoch 10, loss: 2.7726956199391273
Epoch 11, loss: 2.7417409169192406
Epoch 12, loss: 2.7453029276594947
Epoch 13, loss: 2.7491350241058963
Epoch 14, loss: 2.7482723520085433
Epoch 15, loss: 2.740340697713246
Epoch 16, loss: 2.7294832435819307
Epoch 17, loss: 2.7186462237697895
Epoch 18, loss: 2.710459988714808
Epoch 19, loss: 2.764156596139263
Epoch 20, loss: 2.6993432092181644
Epoch 21, loss: 2.7540342349439855
Epoch 22, loss: 2.6866429232731046
Epoch 23, loss: 2.7833720350550326
Epoch 24, loss: 2.767252426166668
Epoch 25, loss: 2.7451865859598366
Epoch 26, loss: 2.67262220269529
Epoch 27, loss: 2.649020034259058
Epoch 28, loss: 2.6320817775480263
Epoch 2

array([[0.49847481],
       [0.50032081],
       [0.62001194]])

## Experiment 2
Data weather

In [4]:
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Loading data
data, meta = arff.loadarff('weather.arff')
data_x = [list(d)[:4] for d in data]
data_y = [[d[4]] for d in data]

# encoding data
le = LabelEncoder()
data_y = le.fit_transform(data_y)

ct = ColumnTransformer([('ohe', OneHotEncoder(), [0, 3]),], remainder= 'passthrough')
data_x = ct.fit_transform(data_x)

# splitting data into train and test data
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size= 0.1)

/home/irfanariq/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
ann = ANN(layer_size=[3, 2, 1], momentum= 0.9, lr= 1e-3, nb_epoch= 30, batch_size=1)
ann.fit(train_x, train_y)
pred = ann.predict(test_x)

print("Prediction: {}".format(pred))
print("Target: {}".format(test_y))

Epoch 0, loss: 12.059046394262744
Epoch 1, loss: 11.68939150080578
Epoch 2, loss: 10.378305546879062
Epoch 3, loss: 10.07006945698248
Epoch 4, loss: 9.838516836503121
Epoch 5, loss: 9.681686196019818
Epoch 6, loss: 9.505369454313882
Epoch 7, loss: 9.399127625280794
Epoch 8, loss: 9.269894069324353
Epoch 9, loss: 9.143821571068832
Epoch 10, loss: 9.069138242234938
Epoch 11, loss: 8.955953909302638
Epoch 12, loss: 8.88657223842729
Epoch 13, loss: 8.838895805761455
Epoch 14, loss: 8.76884030980719
Epoch 15, loss: 8.71928608604832
Epoch 16, loss: 8.63939516584853
Epoch 17, loss: 8.61485338989728
Epoch 18, loss: 8.561612256044578
Epoch 19, loss: 8.524981995399504
Epoch 20, loss: 8.503515754240041
Epoch 21, loss: 8.470319659276749
Epoch 22, loss: 8.439625005465995
Epoch 23, loss: 8.413756578654064
Epoch 24, loss: 8.389274275575543
Epoch 25, loss: 8.386143754508057
Epoch 26, loss: 8.353703186016338
Epoch 27, loss: 8.342410127940393
Epoch 28, loss: 8.31260869806666
Epoch 29, loss: 8.3279183713